# Итоговый проект по курсу Автоматическая Обработка Естественного Языка
#### Выполнили: Женя Егорова, Таня Казакова, Ксюша Кошкина, Настя Чижикова

Корпус текстов собран из постов в пабликах Вконтакте, посвященных феминизму. Тексты выкачаны с помощью VK Api, на начальном этапе взято больше 100 постов длиной не менее 100 слов.

Паблики: https://vk.com/socfem, https://vk.com/radfemtranslations

from datetime import datetime
import requests`
from razdel import tokenize, sentenize

In [ ]:
pip install natasha

     |████████████████████████████████| 34.4MB 100kB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 8.2MB 25.0MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26100 sha256=4e8e4adbfcffaa208fdd372d14d66df2f3aad3c2eba901bf4e980d6733986db2
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
import re

In [ ]:
pip install pymorphy2

In [ ]:
pip install nltk

In [ ]:
pip install razdel

In [ ]:
pip install pymorphy2

In [48]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
from razdel import tokenize, sentenize
from string import punctuation
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import re
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
TOKEN = '2030e8202030e8202030e820cf2041e16f220302030e8207ea4b43071db099fb761cad2'
groups = [-182023900, -46661081]

In [ ]:
posts = []
for group_id in groups:
  data = requests.get(
      'https://api.vk.com/method/wall.get', 
       params={
            "owner_id": group_id,
            "count": 100,
            "v":"5.92",
            "access_token": TOKEN
            }
      ).json()
  group_posts = data['response']['items']

  '''Метаинформация для каждого поста: 
  паблик, дата публикации и id поста, чтобы не потерять впоследствии
  отдельные предложения
  '''

  for p in group_posts:
    if len(p['text'].split()) > 99:
      post_url = f"https://vk.com/public{group_id * -1}?w=wall{group_id}_{p['id']}"
      post_text = p['text']
      date = str(datetime.fromtimestamp(p['date']))

      sentences_list = [_.text for _ in list(sentenize(post_text)) if not _.text.startswith('#') ]
      for sent in sentences_list:
        posts.append((post_url, date, sent.replace('\n', ' ')))  

In [ ]:
len(posts)

3468

In [ ]:
posts

[]

Сохраняем в tsv-файл (вам не надо)

In [ ]:
'''from google.colab import files

with open('sentences.tsv', 'w', encoding='utf-8') as f:
  for p in posts:
    f.write('\t'.join(p) + '\n')

files.download('sentences.tsv')'''

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Вот это вам надо

In [ ]:
sent_df.loc[1474].Sentence

'С 2017 года мы жили вместе, практически гражданским браком, и нам было великолепно, это была настоящая любовь, разница в возрасте вообще не чувствовалась.\nhttps://vk.com/public46661081?w=wall-46661081_76353\t2020-10-13 14:30:03\tРешили, что у нас будет сын и дочка.\nhttps://vk.com/public46661081?w=wall-46661081_76353\t2020-10-13 14:30:03\tИ тут в нашей жизни появилось обстоятельство.\nhttps://vk.com/public46661081?w=wall-46661081_76353\t2020-10-13 14:30:03\tНет, это было чудовище, - театрально повышая тон, заявил подсудимый.\nhttps://vk.com/public46661081?w=wall-46661081_76353\t2020-10-13 14:30:03\t- Страшное чудовище, которое сломало всю нашу жизнь!.'

### Предобработка предложений

In [ ]:
sent_df['Clean_sentence'] = sent_df['Sentence'].apply(preprocess)

Датафрейм униграмм

(код для pos-обработки взяла у Тани)

In [ ]:
def get_pos_lem(word):
  analiz = morph.parse(word)
  return analiz[0].normal_form, analiz[0].tag.POS

In [ ]:
from tqdm.auto import tqdm

In [ ]:
uni_df = pd.DataFrame(columns=['Token', 'Lemma', 'POS', 'Sent_ids'])

for sent_id, sent in tqdm(zip(sent_df.index, sent_df.Clean_sentence)):
    for word in sent.split():
        if word not in uni_df.Token.values:
          lem, pos = get_pos_lem(word) 
          uni_df = uni_df.append({'Token': word,
                          'Lemma': lem,
                          'POS': str(pos),
                          'Sent_ids': str(sent_id)
                          }, ignore_index=True)
        else:
          #ids = uni_df[uni_df.Token == word].Sent_ids.values.tolist()
          uni_df.loc[uni_df.Token == word, 'Sent_ids'] += f' {sent_id}'

In [61]:
def fix_ids(str_list):
  return [int(i) for i in list(set(str_list.split()))]

In [59]:
uni_df['Ids'] = uni_df['Sent_ids'].apply(fix_ids)


In [ ]:
from google.colab import files
uni_df.to_csv('uni_df.tsv', sep='\t', index=False)
files.download('uni_df.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Дата фрейм биграмм


In [ ]:
bi_df = pd.DataFrame(columns=['bi_token', 'uni_first_token', 'uni_first_lemma', 'uni_first_POS', 'uni_second_token', 'uni_second_lemma', 'uni_second_POS', 'Sent_ids'])
token_values = []
for sent_id, sent in tqdm(zip(sent_df.index, sent_df.Clean_sentence)):
  list_words = [word.strip(punctuation) for word in sent.split()]
  text_list = [' '.join(list_words)]
  bigrams = [b for l in text_list for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
  for bigram in bigrams:
    if bigram not in token_values:
      bi_lem = [get_pos_lem(word) for word in bigram]
      bi_df = bi_df.append({'bi_token': bigram,
                          'uni_first_token': bigram[0],
                          'uni_first_lemma': bi_lem[0][0],
                          'uni_first_POS': bi_lem[0][1],
                          'uni_second_token': bigram[1],
                          'uni_second_lemma': bi_lem[1][0], 
                          'uni_second_POS': bi_lem[1][1],
                          'Sent_ids': str(sent_id)
                          }, ignore_index=True)
      token_values.append(bigram)
    else:
      bi_df.loc[bi_df.bi_token == bigram, 'Sent_ids'] += f' {sent_id}'
   

In [ ]:
bi_df['Ids'] = bi_df['Sent_ids'].apply(fix_ids)

In [ ]:
bi_df

,bi_token,uni_first_token,uni_first_lemma,uni_first_POS,uni_second_token,uni_second_lemma,uni_second_POS,Sent_ids,Ids
0,"(в, июне)",в,в,PREP,июне,июнь,NOUN,0 861,"[0, 861]"
1,"(июне, этого)",июне,июнь,NOUN,этого,это,NPRO,0,[0]
2,"(этого, года)",этого,это,NPRO,года,год,NOUN,0 473 1169 1672 1756 3155,"[0, 473, 1756, 1672, 1169, 3155]"
3,"(года, девушка)",года,год,NOUN,девушка,девушка,NOUN,0,[0]
4,"(девушка, по)",девушка,девушка,NOUN,по,по,PREP,0,[0]
...,...,...,...,...,...,...,...,...,...
41739,"(устанавливает, датчики)",устанавливает,устанавливать,VERB,датчики,датчик,NOUN,3466,[3466]
41740,"(датчики, и)",датчики,датчик,NOUN,и,и,CONJ,3466,[3466]
41741,"(затем, программирует)",затем,затем,ADVB,программирует,программировать,VERB,3466,[3466]
41742,"(программирует, их)",программирует,программировать,VERB,их,они,NPRO,3466,[3466]


In [ ]:
from google.colab import files
bi_df.to_csv('bi_df.tsv', sep='\t', index=False)
files.download('bi_df.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Датафрейм триграмм

In [ ]:
tri_df = pd.DataFrame(columns=['tri_token', 'uni_first_token', 'uni_first_lemma', 'uni_first_POS', 'uni_second_token', 'uni_second_lemma', 'uni_second_POS','uni_third_token','uni_third_lemma','uni_third_POS', 'Sent_ids'])
token_values = []
for sent_id, sent in tqdm(zip(sent_df.index, sent_df.Clean_sentence)):
    list_words = [word.strip(punctuation) for word in sent.split()]
    text_list = [' '.join(list_words)]
    trigrams = [b for l in text_list for b in zip(l.split(" ")[:-1], l.split(" ")[1:],l.split(' ')[2:])]
    for trigram in trigrams:
        if trigram not in token_values:
            tri_lem = [get_pos_lem(word) for word in trigram]
            tri_df = tri_df.append({'tri_token': trigram,
                                'uni_first_token': trigram[0],
                                'uni_first_lemma': tri_lem[0][0],
                                'uni_first_POS': tri_lem[0][1],
                                'uni_second_token': trigram[1],
                                'uni_second_lemma': tri_lem[1][0], 
                                'uni_second_POS': tri_lem[1][1],
                                'uni_third_token': trigram[2],
                                'uni_third_lemma': tri_lem[2][0], 
                                'uni_third_POS': tri_lem[2][1],
                                'Sent_ids': str(sent_id)
                          }, ignore_index=True)
            token_values.append(trigram)
        else:
            tri_df.loc[tri_df.tri_token == trigram, 'Sent_ids'] += f' {sent_id}'

In [ ]:
tri_df['Ids'] = tri_df['Sent_ids'].apply(fix_ids)

In [ ]:
tri_df

,tri_token,uni_first_token,uni_first_lemma,uni_first_POS,uni_second_token,uni_second_lemma,uni_second_POS,uni_third_token,uni_third_lemma,uni_third_POS,Sent_ids,Ids
0,"(в, июне, этого)",в,в,PREP,июне,июнь,NOUN,этого,это,NPRO,0,[0]
1,"(июне, этого, года)",июне,июнь,NOUN,этого,это,NPRO,года,год,NOUN,0,[0]
2,"(этого, года, девушка)",этого,это,NPRO,года,год,NOUN,девушка,девушка,NOUN,0,[0]
3,"(года, девушка, по)",года,год,NOUN,девушка,девушка,NOUN,по,по,PREP,0,[0]
4,"(девушка, по, имени)",девушка,девушка,NOUN,по,по,PREP,имени,имя,NOUN,0,[0]
...,...,...,...,...,...,...,...,...,...,...,...,...
46753,"(устанавливает, датчики, и)",устанавливает,устанавливать,VERB,датчики,датчик,NOUN,и,и,CONJ,3466,[3466]
46754,"(датчики, и, затем)",датчики,датчик,NOUN,и,и,CONJ,затем,затем,ADVB,3466,[3466]
46755,"(и, затем, программирует)",и,и,CONJ,затем,затем,ADVB,программирует,программировать,VERB,3466,[3466]
46756,"(затем, программирует, их)",затем,затем,ADVB,программирует,программировать,VERB,их,они,NPRO,3466,[3466]


In [ ]:
from google.colab import files
tri_df.to_csv('tri_df.tsv', sep='\t', index=False)
files.download('tri_df.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sent_df = pd.read_csv('https://raw.githubusercontent.com/nastyachizhikova/NLP_3rdyear/main/sentences.tsv', sep='\t', header=None, names=['URL', 'Date', 'Sentence']) 

In [ ]:
def preprocess(sent):
  tokens = [_.text.strip('-') for _ in list(tokenize(str(sent).lower())) if not re.search('[^а-яА-ЯёЁ\-]', _.text)]
  return ' '.join(tokens)

In [ ]:
sent_df['Clean_sentence'] = sent_df['Sentence'].apply(preprocess)

In [49]:
uni_df = pd.read_csv('https://raw.githubusercontent.com/tbkazakova/autobr_femcorpus/main/uni_df.tsv', sep='\t', header=0, names=['Token1', 'Lemma1', 'POS1', 'Sent_ids'], index_col=False) 

In [50]:
bi_df = pd.read_csv('https://raw.githubusercontent.com/tbkazakova/autobr_femcorpus/main/bi_df.tsv', sep='\t', header=0, names=['bi_token', 'Token1', 'Lemma1', 'POS1', 'Token2', 'Lemma2', 'POS2', 'Sent_ids'], index_col=False) 

In [51]:
tri_df = pd.read_csv('https://raw.githubusercontent.com/tbkazakova/autobr_femcorpus/main/tri_df.tsv', sep='\t', header=0, names=['tri_token', 'Token1', 'Lemma1', 'POS1', 'Token2', 'Lemma2', 'POS2', 'Token3', 'Lemma3', 'POS3', 'Sent_ids'], index_col=False) 

## ПОИСК

In [69]:
POStags = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'PRTF', 'PRTS', 'GRND', 'NUMR', 'ADVB', 'NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ', 'None']

def make_condition(req, index):
    if '+' in req:
        req = req.split('+')
        cond = f'Token{index} == "{req[0]}" and POS{index} == "{req[1]}"'
    elif req.startswith('"'):
        req = req.lower()
        req = req.replace('"', '')
        cond = f'Token{index} == "{req}"'
    elif req in POStags:
        cond = f'POS{index} == "{req}"'
    else:
        req = req.lower()
        req = morph.parse(req)[0].normal_form
        cond = f'Lemma{index} == "{req}"'
    return cond

def get_sentences(req, df):
    conditions = []
    for i, w in enumerate(req.split()):
        conditions.append(make_condition(w, i+1))
    condition = ' and '.join(conditions)
    
    print(condition)
    sent_ids = fix_ids(' '.join(df.query(condition)['Sent_ids'].values.tolist()))
    sentences = sent_df.loc[sent_ids]
    
    return sentences

In [63]:
def get_answer(req):
    if len(req.split()) == 1:
        res = get_sentences(req, uni_df)
    elif len(req.split()) == 2:
        res = get_sentences(req, bi_df)
    elif len(req.split()) == 3:
        res = get_sentences(req, tri_df)
    else:
        print('Запрос в неправильном формате.')
        
    return res

In [71]:
get_answer('травля+NOUN') 

Token1 == "травля" and POS1 == "NOUN"


,URL,Date,Sentence,Clean_sentence
1500,https://vk.com/public46661081?w=wall-46661081_...,2020-10-13 14:30:03,"Дальше - травля, мою парадную стали расписыват...",дальше травля мою парадную стали расписывать ...
90,https://vk.com/public182023900?w=wall-18202390...,2020-07-01 15:49:00,"Склоки, распускание слухов и травля утомляют.",склоки распускание слухов и травля утомляют
202,https://vk.com/public182023900?w=wall-18202390...,2020-04-20 16:33:18,"С этой дружелюбной статьи, кстати, начались тр...",с этой дружелюбной статьи кстати начались трав...
